In [1]:
import tensorflow as tf
import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import tensorflow.keras.metrics as Metrics
import tensorflow.keras.utils as Utils
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
from tensorflow import keras
import matplotlib.pyplot as plot
import numpy as np
from sklearn.metrics import confusion_matrix as CM
from random import randint
from IPython.display import SVG
import matplotlib.gridspec as gridspec

Using TensorFlow backend.


In [2]:
Images = np.load('./dataset/Image_120.npy') #converting the list of images to numpy array.
Labels = np.load('./dataset/Label_120.npy')

In [3]:
print("Shape of Images:",Images.shape)
print("Shape of Labels:",Labels.shape)

Shape of Images: (720, 128, 128, 3)
Shape of Labels: (720,)


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
len(Images)

720

In [6]:
# Images = (Images / 255)

In [7]:
Images.dtype

dtype('uint8')

In [8]:
Images = Images.astype('float32')
Images.dtype, Labels.dtype

(dtype('float32'), dtype('int64'))

In [9]:
names = {0: 'Tomato_H', 1: 'Tomato_D01', 2: 'Tomato_D04', 3: 'Tomato_D05', 
             4: 'Tomato_D07', 5: 'Tomato_D08', 6: 'Tomato_D09', 7: 'Tomato_P03', 
             8: 'Tomato_P05'}

In [10]:
X_train, X_test ,y_train, y_test = train_test_split(Images, Labels,test_size=0.3333, stratify=Labels, random_state=1, shuffle=True)

In [11]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
np.bincount(y_train),np.bincount(y_test)

(480, 128, 128, 3) (240, 128, 128, 3) (480,) (240,)


(array([80, 80, 80, 80, 80, 80]), array([40, 40, 40, 40, 40, 40]))

In [12]:
y_test[0:20]

array([3, 4, 4, 0, 3, 0, 0, 5, 4, 4, 1, 0, 1, 0, 3, 2, 3, 5, 2, 2])

In [13]:
X_train[0:1].shape

(1, 128, 128, 3)

In [14]:
from keras.applications.resnet50 import ResNet50
IMG_SHAPE = X_train[0].shape

base_model = ResNet50(input_shape=IMG_SHAPE,
                                            include_top=False,
                                            pooling='max',   
                                            weights='imagenet')













A local file was found, but it seems to be incomplete or outdated because the md5 file hash does not match the original value of a268eb855778b3df3c7506639542a6af so we will re-download the data.
94658560/94653016 [==============================] - 14s 0us/step


In [15]:
test_batch = tf.convert_to_tensor(X_train[0:8])
test_batch

<tf.Tensor 'Const:0' shape=(8, 128, 128, 3) dtype=float32>

In [16]:
feature_batch = base_model(test_batch)
print(feature_batch.shape)

(8, 2048)


In [17]:
# base_model.trainable = False

In [18]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [19]:
# global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# feature_batch_average = global_average_layer(feature_batch)
# print(feature_batch_average.shape)

In [21]:
prediction_layer = keras.layers.Dense(6, activation='softmax')
# prediction_batch = prediction_layer(feature_batch)
# print(prediction_batch.shape)

In [23]:
model = tf.keras.Sequential([
  base_model,
  prediction_layer
])
# for layer in model.layers:
#     if hasattr(layer, 'moving_mean') and hasattr(layer, 'moving_variance'):
#         layer.trainable = True
#         K.eval(K.update(layer.moving_mean, K.zeros_like(layer.moving_mean)))
#         K.eval(K.update(layer.moving_variance, K.zeros_like(layer.moving_variance)))
#     else:
#         layer.trainable = False
        
# model.add(prediction_layer)

TypeError: The added layer must be an instance of class Layer. Found: <keras.engine.training.Model object at 0x7fc78e7f0f90>

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.SGD(lr=base_learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# from sklearn.utils import class_weight

# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(y_train),
#                                                  y_train)
# class_weights

In [ ]:
# class_weight = {}

# for i in range(9):
#     class_weight[i] = class_weights[i]
    
# class_weight

In [ ]:
# class_weight = [np.sqrt(np.sqrt(i)) for i in class_weights]
# class_weight

In [ ]:
X_train_1, X_val, y_train_1, y_val = train_test_split(X_train, y_train, test_size=0.5, stratify=y_train, shuffle=True, random_state=1 )

In [ ]:
y_val.shape, np.bincount(y_val), y_val.dtype, X_train_1.dtype

In [ ]:
model.summary()

In [ ]:
model.trainable_variables

In [ ]:
# trained = model.fit(X_train_1, y_train_1, epochs=100, batch_size=32, validation_data=(X_val,y_val), class_weight = class_weight)
trained = model.fit(X_train_1, y_train_1, epochs=100, batch_size=4, validation_data=(X_val,y_val))

In [ ]:
plot.plot(trained.history['acc'])
plot.plot(trained.history['val_acc'])
plot.title('Model accuracy')
plot.ylabel('Accuracy')
plot.xlabel('Epoch')
plot.legend(['Train', 'Valid'], loc='upper left')
plot.show()

plot.plot(trained.history['loss'])
plot.plot(trained.history['val_loss'])
plot.title('Model loss')
plot.ylabel('Loss')
plot.xlabel('Epoch')
plot.legend(['Train', 'Valid'], loc='upper left')
plot.show()

In [ ]:
# test_images,test_labels = get_images('./test/')
# test_images = np.array(test_images)
# test_labels = np.array(test_labels)
model.evaluate(X_test,y_test, verbose=1)

In [ ]:
model.save('90per_2root.h5')